In [1]:
from numpy.random import default_rng
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

rng = default_rng()

mean = 0.
sd = 0.01

eps = 0.1
steps = 200000

In [3]:
result = pd.DataFrame(columns=['avg','const'])

q_true = [0.]*10
q1 = [0.]*10
n1 = [0.]*10
q2 = [0.]*10
alfa_const = 0.1

for st in tqdm(range(steps)):
    # true
    for i in range(10):
        q_true[i] += rng.normal(mean,sd)

    # sample-average
    i = rng.integers(0,10) if rng.random() < eps else np.argmax(q1)
    result.loc[st,'avg'] = q_true[i]
    n1[i] += 1
    q1[i] += (q_true[i] - q1[i]) / n1[i]

    # const
    i = rng.integers(0,10) if rng.random() < eps else np.argmax(q2)
    result.loc[st,'const'] = q_true[i]
    q2[i] += alfa_const * (q_true[i] - q2[i])

 26%|██▌       | 51102/200000 [01:56<05:39, 438.29it/s] 


KeyboardInterrupt: 